<a href="https://colab.research.google.com/github/trvoid/llm-study/blob/main/bert/getting_started_with_distilbert_for_qa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DistilBERT for QA 시작하기

이 실습은 아래 문서의 내용을 토대로 진행하였습니다.

* [Fine-Tuning DistilBERT for Question Answering](https://machinelearningmastery.com/fine-tuning-distilbert-for-question-answering/), By Muhammad Asad Iqbal Khan on March 29, 2025

## 1. 해결하고자 하는 문제

다음과 같은 유형의 문제를 언어 모델을 사용하여 풀고자 합니다.

* 지문(context): Tom sits on a bench.
* 질문(question): Where does Tom sit?
* 답변(answer): bench (지문에서 첫번째 문자의 위치를 0이라고 할 때 답변의 시작 위치는 14, 끝 위치는 19)

In [ ]:
context = "Tom sits on a bench."
answer = "bench"

start_char = context.find(answer)
end_char = start_char + len(answer)
print(f"start_char={start_char}, end_char={end_char}")

start_char=14, end_char=19


주어진 지문으로부터 질문에 답변하는 방식에 따라 아래 두 가지로 구분할 수 있습니다.

* 추출형 질의응답 (Extractive Question Answering): 주어진 지문 내에서 질문에 대한 답변에 해당하는 부분을 그대로 찾아 추출하는 방식
* 생성형 질의응답 (Abstractive Question Answering): 주어진 지문의 내용을 이해하고 요약하거나 재구성하여 질문에 대한 답변을 새롭게 생성하는 방식

여기서 다루는 문제는 추출형 질의응답에 해당합니다.

## 2. 문제 해결 방안

언어 모델 훈련에 사용할 데이터를 아래의 형식에 맞추어 준비합니다.

* 입력 데이터: 질문과 지문을 토큰화하고 이어붙인 배열
  * "[CLS] *question* [SEP] *context* [SEP]"
* 정답 데이터: 입력 데이터에서 답변의 시작 토큰과 끝 토큰의 위치 정보
  * 입력 데이터에서 답변의 시작 토큰 위치
  * 입력 데이터에서 답변의 끝 토큰 위치

위 형식의 데이터를 대상으로 훈련하는 과정은 다음과 같습니다.

* 모델과 데이터셋 선택
  * 언어 이해를 목적으로 훈련된 기본 모델 선택
  * 질의응답 작업을 위해 미세조정 훈련을 수행할 모델 선택
  * 질의응답 미세조정 훈련에 사용할 데이터셋 선택
* 미세조정 훈련
  1. 미세조정 모델로 정답을 예측하고 손실 계산
  2. 미세조정 모델 파라미터 갱신

이 실습에서 사용할 모델과 데이터셋은 다음과 같습니다.

1. [DistilBERT](https://huggingface.co/docs/transformers/en/model_doc/distilbert):기본 모델 (단어에 대한 단순 임베딩이 아니라 맥락을 고려한 임베딩 수행)
2. [DistilBertForQuestionAnswering](https://huggingface.co/docs/transformers/en/model_doc/distilbert?usage=Pipeline#transformers.DistilBertForQuestionAnswering): DistilBERT 모델에 질의응답 층을 추가한 것으로서 질의응답 미세조정 훈련을 위한 모델
3. [SQuAD](https://huggingface.co/datasets/rajpurkar/squad): 질의응답 미세조정 훈련을 위한 데이터셋

## 3. 데이터셋 적재

`datasets` 라이브러리를 설치합니다.

In [ ]:
!pip install datasets

`squad` 데이터셋을 적재합니다.

In [ ]:
from datasets import load_dataset

# Load the SQuAD dataset
dataset = load_dataset("squad")

`dataset`의 구조를 출력해서 확인합니다.

In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})


"train" 항목의 첫번째 데이터를 출력해서 실제로 어떤 값을 가지고 있는지 확인합니다.

In [ ]:
import json

print(json.dumps(dataset["train"][0], indent=4))

{
    "id": "5733be284776f41900661182",
    "title": "University_of_Notre_Dame",
    "context": "Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend \"Venite Ad Me Omnes\". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.",
    "question": "To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?",
    "answers": {
        "text": [
            "Saint Bernadette Soubirous"
        ],
        "answer_start": [

## 4. 토크나이저와 모델 적재

`transformers` 라이브러리를 설치합니다.

In [ ]:
!pip install transformers

기본 모델 `distilbert-base-uncased`를 지정하고 `DistilBertTokenizerFast` 클래스를 사용하여 토크나이저를, `DistilBertForQuestionAnswering` 클래스를 사용하여 모델을 적재합니다.

In [ ]:
from transformers import DistilBertTokenizerFast, DistilBertForQuestionAnswering

# Load tokenizer and model
base_model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizerFast.from_pretrained(base_model_name)
model = DistilBertForQuestionAnswering.from_pretrained(base_model_name)

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 5. 토크나이저 테스트

훈련 데이터 준비 과정에 대한 이해를 돕기 위하여 간단한 예제로 토크나이저를 테스트해 봅니다.

In [ ]:
questions = ["Where does Tom sits?"]
contexts = ["Tom sits on a bench."]

inputs = tokenizer(
        questions,
        contexts,
        max_length=15,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
)

토크나이저 실행 결과인 `inputs`의 유형과 키 항목들을 출력합니다.

In [ ]:
print(type(inputs))
print(inputs.keys())

<class 'transformers.tokenization_utils_base.BatchEncoding'>
dict_keys(['input_ids', 'attention_mask', 'offset_mapping'])


각각의 키 항목들에 해당하는 값을 출력하면 다음과 같습니다.

In [ ]:
print("input_ids: ", end="")
print(inputs["input_ids"])

print("attention_mask: ", end="")
print(inputs["attention_mask"])

print("offset_mapping: ", end="")
print(inputs["offset_mapping"])

input_ids: [[101, 2073, 2515, 3419, 7719, 1029, 102, 3419, 7719, 2006, 1037, 6847, 1012, 102, 0]]
attention_mask: [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]]
offset_mapping: [[(0, 0), (0, 5), (6, 10), (11, 14), (15, 19), (19, 20), (0, 0), (0, 3), (4, 8), (9, 11), (12, 13), (14, 19), (19, 20), (0, 0), (0, 0)]]


`input_ids`에 해당하는 값을 토큰 문자열로 변환해서 보면 각각의 키 항목들의 의미를 좀 더 쉽게 이해할 수 있습니다.

In [ ]:
token_strs = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
print("token_strs: ", end="")
print(token_strs)

token_strs: ['[CLS]', 'where', 'does', 'tom', 'sits', '?', '[SEP]', 'tom', 'sits', 'on', 'a', 'bench', '.', '[SEP]', '[PAD]']


`token_strs`을 살펴 보면 답변에 해당하는 토큰 'bench'의 시작 위치와 끝 위치가  각각 11임을 알 수 있습니다.

## 6. 토큰 배열에서 답변의 시작 토큰 위치와 끝 토큰 위치 찾는 방법

토크나이저 출력 결과 중에서 활용해야 할 정보가 하나 더 있는데 그것은 sequence_ids입니다. 이것으로부터 각각의 토큰이 토크나이저에 입력한 문자열들 중에서 몇 번째 문자열에 해당하는 것인지 파악할 수 있습니다.

* sequence_ids의 값이 0이면 토크나이저의 첫번째 인자(`questions`)로부터 온 토큰
* sequence_ids의 값이 1이면 토크나이저의 두번째 인자(`contexts`)로부터 온 토큰

In [ ]:
print("type(sequence_ids): ", end="")
print(type(inputs.sequence_ids))

print("sequence_ids(0): ", end="")
print(inputs.sequence_ids(0))

type(sequence_ids): <class 'method'>
sequence_ids(0): [None, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, None, None]


이제 토큰 배열에서 답변의 시작 토큰 위치와 끝 토큰 위치를 찾기 위해 필요로 하는 정보는 모두 확보하였습니다.

* `inputs.sequence_ids(0): [None, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, None, None]`
* `inputs["offset_mapping"][0]: [(0, 0), (0, 5), (6, 10), (11, 14), (15, 19), (19, 20), (0, 0), (0, 3), (4, 8), (9, 11), (12, 13), (14, 19), (19, 20), (0, 0), (0, 0)]`
* `start_char: 14`
* `end_char: 19`

이 정보들을 활용하여 답변의 위치를 기계적으로 찾는 과정은 다음과 같습니다.

1. sequence_ids에서 context에 해당하는 토큰들 위치 찾기
  1. 값이 1인 요소들 중에서 첫번째 것의 위치: `7`
  2. 값이 1인 요소들 중에서 마지막 것의 위치: `12`
2. offset_mapping에서 context에 해당하는 토큰들을 대상으로 답변의 시작 위치와 끝 위치에 해당하는 토큰 위치 찾기
  1. `start_char` 값인 14를 포함하는 offset_mapping 튜플의 위치: `11`
  2. `end_char` 값인 19를 포함하는 offset_mapping 튜플의 위치: `11`

답변의 시작 토큰 위치와 끝 토큰의 위치가 훈련 데이터에서 입력 데이터에 대한 정답으로 간주되는 값입니다.

## 7. 훈련 데이터 생성

원본 데이터셋에서 개별 데이터 항목은 아래와 같은 구조를 가집니다.

In [ ]:
examples_like_squad = [
    {
        "context": "Tom sits on a bench.",
        "question": "Where does Tom sit?",
        "answers": {
            "text": ["bench"],
            "answer_start": [14]
        }
    }
]

훈련용 데이터 형식으로 변환하는 전처리 함수는 아래와 같은 구조의 데이터를 인자로 받아서 처리합니다.

In [ ]:
examples_for_preprocess = {
    "context": ["Tom sits on a bench."],
    "question": ["Where does Tom sit?"],
    "answers": [
        {
            "text": ["bench"],
            "answer_start": [14]
        }
    ]
}

데이터셋을 입력으로 받아서 토큰 배열로 만들고 답변의 시작 토큰 위치와 끝 토큰의 위치를 찾아서 훈련용 데이터를 만드는 전처리 함수의 구현은 다음과 같습니다.

In [ ]:
# Tokenize the dataset
def preprocess_function(examples, max_length=384):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offsets in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = start_char + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        context_start = sequence_ids.index(1)
        context_end = len(sequence_ids) - 1 - sequence_ids[::-1].index(1)

        # If the answer is not fully inside the context, label it (0, 0)
        if offsets[context_start][0] > end_char or offsets[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise find the start and end token positions
            idx = context_start
            while idx <= context_end and offsets[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offsets[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

위에서 준비한 테스트용 데이터셋을 사용하여 전처리 함수를 실행해 봅니다. 화면 표시의 편의를 위하여 `max_length`의 값을 15로 지정하였습니다.

In [ ]:
results = preprocess_function(examples_for_preprocess, max_length=15)
print(f'input_ids: {results["input_ids"]}')
print(f'attention_mask: {results["attention_mask"]}')
print(f'start_positions: {results["start_positions"]}')
print(f'end_positions: {results["end_positions"]}')

input_ids: [[101, 2073, 2515, 3419, 4133, 1029, 102, 3419, 7719, 2006, 1037, 6847, 1012, 102, 0]]
attention_mask: [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]]
start_positions: [11]
end_positions: [11]


In [ ]:
# Apply preprocessing to the dataset
tokenized_datasets = dataset.map(preprocess_function,
                                 batched=True,
                                 remove_columns=dataset["train"].column_names)

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

## 8. 미세조정 훈련

이제 SQuAD 데이터셋을 대상으로 DistilBertForQuestionAnswering 모델을 훈련합니다.

In [ ]:
!pip install torch accelerate

In [ ]:
import os
os.environ['WANDB_DISABLED'] = 'true'

from transformers import Trainer, TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
)

# Train the model and save the results
trainer.train()

finetuned_model_path = "./fine-tuned-distilbert-squad"
model.save_pretrained(finetuned_model_path)
tokenizer.save_pretrained(finetuned_model_path)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


NameError: name 'model' is not defined

## 9. 미세조정 훈련 모델로 질의응답 수행

In [ ]:
def predict_answer(tokenizer, model, context, question):
    inputs = tokenizer(question, context, return_tensors="pt")
    input_tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    print(input_tokens)

    with torch.no_grad():
        outputs = model(**inputs)

    answer_start_index = outputs.start_logits.argmax()
    answer_end_index = outputs.end_logits.argmax()

    predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
    output_tokens = tokenizer.convert_ids_to_tokens(predict_answer_tokens)

    return output_tokens

먼저 기본 모델로 질의응답을 수행해 봅니다.

In [ ]:
test_context = "Jim Henson was a nice puppet"
test_question = "Who was Jim Henson?"

base_tokenizer = DistilBertTokenizerFast.from_pretrained(base_model_name)
base_model = DistilBertForQuestionAnswering.from_pretrained(base_model_name)

test_output_tokens = predict_answer(base_tokenizer, base_model, test_context, test_question)
print(f"answer by base model: {test_output_tokens}")

이제 미세조정 모델로 답변을 예측하고 기본 모델 예측 결과와 비교합니다.

In [ ]:
finetuned_tokenizer = DistilBertTokenizerFast.from_pretrained(finetuned_model_path)
finetuned_model = DistilBertForQuestionAnswering.from_pretrained(finetuned_model_path)

test_output_tokens = predict_answer(finetuned_tokenizer, finetuned_model, test_context, test_question)
print(f"answer by finetuned model: {test_output_tokens}")